In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import kruskal
import pymysql

In [ ]:
db_connection = pymysql.connect(
    read_default_file='/home/jupyter/.my.cnf',
    read_default_group='RAnalysis', 
    db = 'actin_personalization'
)

query = """  
SELECT 
    * 
FROM 
    knownPalliativeTreatments 
"""

df = pd.read_sql(query, db_connection)

db_connection.close()

df.head()

# Assumption 5

In [ ]:
df = df[df['tumorIncidenceYear'] != 2022]

pfs_by_year = df.groupby('tumorIncidenceYear')['observedPfsDays'].median()

years = df['tumorIncidenceYear'].unique()
pfs_data = [df[df['tumorIncidenceYear'] == year]['observedPfsDays'].dropna() for year in years]
kruskal_test = kruskal(*pfs_data)

print("Kruskal-Wallis test results:", kruskal_test)

plt.figure(figsize=(10, 6))
sns.boxplot(x='tumorIncidenceYear', y='observedPfsDays', data=df)
plt.title("PFS Distribution by Start Year")
plt.xticks(rotation=45)
plt.show()

In [ ]:
os_by_year = df.groupby('tumorIncidenceYear')['observedOsFromTreatmentStartDays'].median()

years = df['tumorIncidenceYear'].unique()
os_data = [df[df['tumorIncidenceYear'] == year]['observedOsFromTreatmentStartDays'].dropna() for year in years]
kruskal_test = kruskal(*os_data)

print("Kruskal-Wallis test results:", kruskal_test)

plt.figure(figsize=(10, 6))
sns.boxplot(x='tumorIncidenceYear', y='observedOsFromTreatmentStartDays', data=df)
plt.title("OS Distribution by Start Year")
plt.xticks(rotation=45)
plt.show()

In [ ]:
treatments = df['systemicTreatmentPlan'].dropna().unique()

num_cols = 4
num_rows = (len(treatments) + num_cols - 1) // num_cols  

fig, axes = plt.subplots(nrows=num_rows, ncols=num_cols, figsize=(18, num_rows * 4))
fig.tight_layout(pad=5.0)
axes = axes.flatten()

pfs_kruskal_results = []

for i, treatment in enumerate(treatments):
    treatment_data = df[df['systemicTreatmentPlan'] == treatment]
    pfs_data = [treatment_data[treatment_data['tumorIncidenceYear'] == year]['observedPfsDays'].dropna() for year in years]
    
    filtered_pfs_data = [data for data in pfs_data if len(data) > 0]
    
    if len(filtered_pfs_data) > 1:
        kruskal_test = kruskal(*filtered_pfs_data)
        pfs_kruskal_results.append([treatment, kruskal_test.statistic, kruskal_test.pvalue])
        
        # Plot the boxplot for each treatment in the respective subplot
        sns.boxplot(x='tumorIncidenceYear', y='observedPfsDays', data=treatment_data, ax=axes[i])
        axes[i].set_title(f"PFS Distribution by Year for {treatment}")
        axes[i].set_xticks(axes[i].get_xticks())
        axes[i].set_xticklabels(axes[i].get_xticklabels(), rotation=45)

for j in range(i+1, len(axes)):
    fig.delaxes(axes[j])

plt.show()

In [ ]:
pfs_kruskal_results_df = pd.DataFrame(pfs_kruskal_results, columns=['Treatment', 'Kruskal-Wallis Statistic', 'p-value'])

pfs_kruskal_results_df

In [ ]:
num_cols = 4
num_rows = (len(treatments) + num_cols - 1) // num_cols  

fig, axes = plt.subplots(nrows=num_rows, ncols=num_cols, figsize=(18, num_rows * 4))
fig.tight_layout(pad=5.0)
axes = axes.flatten()

os_kruskal_results = []

for i, treatment in enumerate(treatments):
    treatment_data = df[df['systemicTreatmentPlan'] == treatment]
    os_data = [treatment_data[treatment_data['tumorIncidenceYear'] == year]['observedOsFromTreatmentStartDays'].dropna() for year in years]
    
    filtered_os_data = [data for data in os_data if len(data) > 0]
    
    if len(filtered_pfs_data) > 1:
        kruskal_test = kruskal(*filtered_os_data)
        os_kruskal_results.append([treatment, kruskal_test.statistic, kruskal_test.pvalue])
        
        # Plot the boxplot for each treatment in the respective subplot
        sns.boxplot(x='tumorIncidenceYear', y='observedOsFromTreatmentStartDays', data=treatment_data, ax=axes[i])
        axes[i].set_title(f"OS Distribution by Year for {treatment}")
        axes[i].set_xticks(axes[i].get_xticks())
        axes[i].set_xticklabels(axes[i].get_xticklabels(), rotation=45)

for j in range(i+1, len(axes)):
    fig.delaxes(axes[j])

plt.show()

In [ ]:
os_kruskal_results_df = pd.DataFrame(os_kruskal_results, columns=['Treatment', 'Kruskal-Wallis Statistic', 'p-value'])

os_kruskal_results_df

# Assumption 6

In [ ]:
event_counts = df.pivot_table(index='systemicTreatmentPlan', 
                              columns='hadProgressionEvent',
                              values='episodeId', 
                              aggfunc='count', 
                              fill_value=0)

event_counts['censored_percentage'] = event_counts[0] / (event_counts[1] + event_counts[0]) * 100

event_counts['total_events'] = event_counts[1] + event_counts[0]
event_counts['censor_events'] = event_counts[0]
event_counts['progression_events'] = event_counts[1]

plt.figure(figsize=(10, 6))
bars = event_counts['censored_percentage'].plot(kind='bar')
plt.title("Censoring Percentage by Treatment Group (Using hadProgressionEvent)")
plt.ylabel("Censored Percentage")
plt.xticks(rotation=45, ha="right") 
plt.tight_layout()

plt.show()

In [ ]:
event_counts = df.pivot_table(index='systemicTreatmentPlan', 
                              columns='hadSurvivalEvent',
                              values='episodeId', 
                              aggfunc='count', 
                              fill_value=0)

event_counts.head()

event_counts['censored_percentage'] = event_counts[0] / (event_counts[1] + event_counts[0]) * 100

event_counts['total_events'] = event_counts[1] + event_counts[0]
event_counts['censor_events'] = event_counts[0]
event_counts['progression_events'] = event_counts[1]

plt.figure(figsize=(10, 6))
bars = event_counts['censored_percentage'].plot(kind='bar')
plt.title("Censoring Percentage by Treatment Group (Using hadSurvivalEvent)")
plt.ylabel("Censored Percentage")
plt.xticks(rotation=45, ha="right") 
plt.tight_layout()

plt.show()